In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [4]:


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test_supplement.csv', 'train.csv', 'test.csv', 'sample_submission.csv', 'train_sample.csv']

import numpy as np
import pandas as pd
import math
import time
import os.path

import xgboost as xgb

# all attributed = 0 means 0.5 score

def print_duration (start_time, msg):
    print("[%d] %s" % (int(time.time() - start_time), msg))
    start_time = time.time()
    return start_time

def main():
    start_time = time.time()
    # create a xgboost model
    model = xgb.XGBClassifier(objective="binary:logistic",min_child_weight=1, max_depth=5, learning_rate=.1, 
                            n_estimators=1000, n_jobs=-1,gamma=0, nthread=4, scale_pos_weight=1, seed=27)

    
    chunksize = 5000000 
    chunk_idx = 0 
    chunk_max = 2
    for chunk in pd.read_csv('../input/train.csv', chunksize=chunksize):
        chunk['hour'] = pd.to_datetime(chunk.click_time).dt.hour.astype('uint8')
        chunk['day'] = pd.to_datetime(chunk.click_time).dt.day.astype('uint8')
        nb_clicks = chunk.groupby(['ip','hour'])['channel'].count().reset_index()
        nb_clicks.columns = ['ip','hour', 'clicks_by_ip']
        chunk = pd.merge(chunk, nb_clicks, on=['ip','hour'], how='left', sort=False)
        chunk['clicks_by_ip'] = chunk['clicks_by_ip'].astype('uint16') 
        train_X = chunk.as_matrix(columns=['clicks_by_ip','app', 'device', 'os', 'hour', 'day',])
        if chunk_idx > 0: # not load in first run
            model.fit(train_X, chunk['is_attributed'], xgb_model=model)
        else:
            model.fit(train_X, chunk['is_attributed'])
        print("Processed chunk %d" % chunk_idx)
        chunk_idx = chunk_idx + 1
        if chunk_idx >= chunk_max:
            break
        
    # read test data set
    test = pd.read_csv('../input/test.csv')
    test['hour'] = pd.to_datetime(test.click_time).dt.hour.astype('uint8')
    test['day'] = pd.to_datetime(test.click_time).dt.day.astype('uint8')
    nb_clicks = test.groupby(['ip','hour'])['channel'].count().reset_index()
    nb_clicks.columns = ['ip','hour', 'clicks_by_ip']
    test = pd.merge(test, nb_clicks, on=['ip','hour'], how='left', sort=False)
    test['clicks_by_ip'] = test['clicks_by_ip'].astype('uint16')  
    test_X = test.as_matrix(columns=['clicks_by_ip','app', 'device', 'os', 'hour', 'day',])
    start_time = print_duration (start_time, "Finished training, start prediction")   
    # predict the propabilities for binary classes    
    pred = model.predict_proba(test_X)
    start_time = print_duration (start_time, "Finished prediction, start store results")    
    submission = pd.read_csv("../input/sample_submission.csv")
    submission['is_attributed'] = pred[:,1]
    submission.to_csv("submission.csv", index=False)
    start_time = print_duration(start_time, "Finished to store result")
    
    
if __name__ == '__main__':
    main()
    

